# WORLD BANK ETL — 40 QUESTIONS

In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
import logging
from datetime import datetime
import os
from dotenv import load_dotenv

# Setup directories
os.makedirs('data', exist_ok=True)
os.makedirs('cleaned', exist_ok=True)
os.makedirs('logs', exist_ok=True)

# Setup logging
log_file = f'logs/etl_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
logging.basicConfig(filename=log_file, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')
print(f"Log: {log_file}")

Log: logs/etl_20251110_204343.log


In [3]:
load_dotenv()
api_key = os.getenv('API_KEY')
print(f"API Key Loaded: {api_key is not None}")

API Key Loaded: True


## PART 1: EXTRACT (Q1–Q10)

In [7]:
# Q1–Q3: Load CSVs
main_df = pd.read_csv('data/main_data.csv')
meta_df = pd.read_csv('data/metadata_country.csv')
pop_df = pd.read_csv('data/population.csv')

print("Q1–Q3: Loaded files")
for name, df in [('main', main_df), ('meta', meta_df), ('pop', pop_df)]:
    print(f"\n{name.upper()}: {df.shape[0]:,} × {df.shape[1]}")
    display(df.head())
logging.info("Q1–Q3: CSVs loaded")

Q1–Q3: Loaded files

MAIN: 211 × 68


,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2016,2017,2018,2019,2020,2021,2022,2023,Region,IncomeGroup
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.558906e+09,3.103184e+09,3.544708e+09,3.598637e+09,Latin America & Caribbean,High income
1,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426650e+10,1.450216e+10,2.262420e+10,South Asia,Low income
2,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.040000e+11,8.472296e+10,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.891638e+10,2.297768e+10,Europe & Central Asia,Upper middle income
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.324683e+09,3.380602e+09,3.727674e+09,Europe & Central Asia,High income



META: 263 × 4


,Country Code,Region,IncomeGroup,TableName
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFE,East Africa,Upper middle income,Africa Eastern and Southern
2,AFG,South Asia,Low income,Afghanistan
3,AFW,West Africa,Low income,Africa Western and Central
4,AGO,Sub-Saharan Africa,Lower middle income,Angola



POP: 216 × 66


,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,54608,55811,56682,57475,58178,58782,59291,59522,...,103594,104257,104874,105439,105962,106442,106585,106537,106445,106277
1,Afghanistan,AFG,8622466,8790140,8969047,9157465,9355514,9565147,9783147,10010030,...,32716210,33753499,34636207,35643418,36686784,37769499,38972230,40099462,41128771,42239854
2,Angola,AGO,5357195,5441333,5521400,5599827,5673199,5736582,5787044,5827503,...,27128337,28127721,29154746,30208628,31273533,32353588,33428486,34503774,35588987,36684202
3,Albania,ALB,1608800,1659800,1711319,1762621,1814135,1864791,1914573,1965598,...,2889104,2880703,2876101,2873457,2866376,2854191,2837849,2811666,2777689,2745972
4,Andorra,AND,9443,10216,11014,11839,12690,13563,14546,15745,...,71621,71746,72540,73837,75013,76343,77700,79034,79824,80088


In [8]:
# Q4: Dtypes
print("Q4: Data types")
for name, df in [('main', main_df), ('meta', meta_df), ('pop', pop_df)]:
    print(f"\n{name.upper()} dtypes:")
    print(df.dtypes)
logging.info("Q4: Dtypes shown")

Q4: Data types

MAIN dtypes:
Country Name     object
Country Code     object
1960            float64
1961            float64
1962            float64
                 ...   
2021            float64
2022            float64
2023            float64
Region           object
IncomeGroup      object
Length: 68, dtype: object

META dtypes:
Country Code    object
Region          object
IncomeGroup     object
TableName       object
dtype: object

POP dtypes:
Country Name    object
Country Code    object
1960             int64
1961             int64
1962             int64
                 ...  
2019             int64
2020             int64
2021             int64
2022             int64
2023             int64
Length: 66, dtype: object


In [9]:
# Q5: Unique countries
unique = main_df['Country Code'].nunique()
print(f"Q5: Unique countries: {unique}")
logging.info(f"Q5: {unique} unique countries")

Q5: Unique countries: 211


In [10]:
# Q6: Indicator names
print("Q6: No indicator_name in metadata_country.csv — only country metadata")
logging.info("Q6: metadata_country has no indicators")

Q6: No indicator_name in metadata_country.csv — only country metadata


In [11]:
# Q7: Missing values
print("Q7: Missing values")
for name, df in [('main', main_df), ('meta', meta_df), ('pop', pop_df)]:
    miss = df.isnull().sum().sum()
    print(f"  {name}: {miss} total")
logging.info("Q7: Missing values checked")

Q7: Missing values
  main: 2350 total
  meta: 0 total
  pop: 0 total


In [12]:
# Q8: Missing year/value
year_cols = [c for c in main_df.columns if c.isdigit()]
no_data = main_df[year_cols].isnull().all(axis=1).sum()
print(f"Q8: {no_data} rows with no value")
logging.info(f"Q8: {no_data} rows missing all values")

Q8: 0 rows with no value


In [13]:
# Q9: Country code check
invalid = set(main_df['Country Code']) - set(pop_df['Country Code'])
print(f"Q9: {len(invalid)} invalid codes")
logging.info(f"Q9: {len(invalid)} invalid country codes")

Q9: 0 invalid codes


In [14]:
# Q10: main_data is WIDE format
print("Q10: main_data is WIDE (no indicator_code)")
logging.info("Q10: WIDE format confirmed")

Q10: main_data is WIDE (no indicator_code)


## PART 2: CLEAN (Q11–Q19)

In [15]:
# Q11–Q12: Clean values
main_df.replace(['..', '', ' '], np.nan, inplace=True)
year_cols = [c for c in main_df.columns if c.isdigit()]
main_df[year_cols] = main_df[year_cols].astype(float)
print("Q11–Q12: Cleaned values")
logging.info("Q11–Q12: Values cleaned")

Q11–Q12: Cleaned values


In [16]:
# Q13–Q14: Duplicates + trim
before = len(main_df)
main_df.drop_duplicates(inplace=True)
main_df = main_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
print(f"Q13: Removed {before - len(main_df)} duplicates")
logging.info(f"Q13: Removed duplicates")

Q13: Removed 0 duplicates


In [17]:
# Q15–Q17: Standardize + valid years + drop empty
main_df['Country Name'] = main_df['Country Name'].replace({'Kenia': 'Kenya'})
valid = [str(y) for y in range(1960, 2024)]
invalid = [c for c in year_cols if c not in valid]
if invalid: main_df.drop(columns=invalid, inplace=True)
empty = main_df.columns[main_df.isnull().all()]
main_df.drop(columns=empty, inplace=True)
print("Q15–Q17: Cleaned names, years, empty cols")
logging.info("Q15–Q17: Standardized")

Q15–Q17: Cleaned names, years, empty cols


In [18]:
# Q18: Already have codes
print("Q18: Country codes present")
logging.info("Q18: Codes OK")

Q18: Country codes present


In [19]:
# Q19: Save cleaned
main_df.to_csv('cleaned/main_cleaned.csv', index=False)
meta_df.to_csv('cleaned/metadata_country_cleaned.csv', index=False)
pop_df.to_csv('cleaned/population_cleaned.csv', index=False)
print("Q19: Cleaned CSVs saved")
logging.info("Q19: Cleaned files saved")

Q19: Cleaned CSVs saved


## PART 3: TRANSFORM (Q21–Q30)

In [20]:
# Q21–Q22: Melt + merge
id_vars = ['Country Name', 'Country Code', 'Region', 'IncomeGroup']
gdp_long = pd.melt(main_df, id_vars=id_vars, value_vars=year_cols, var_name='year', value_name='gdp')
pop_long = pd.melt(pop_df, id_vars=['Country Name', 'Country Code'], value_vars=year_cols, var_name='year', value_name='population')
merged = gdp_long.merge(pop_long, on=['Country Code', 'Country Name', 'year'], how='left')
merged['year'] = merged['year'].astype(int)
print(f"Q21–Q22: Merged: {merged.shape[0]:,} rows")
logging.info("Q21–Q22: Merged")

Q21–Q22: Merged: 13,504 rows


In [21]:
# Q23: GDP per capita change
merged['gdp_per_capita'] = merged['gdp'] / merged['population']
merged = merged.sort_values(['Country Code', 'year'])
merged['gdp_pc_change'] = merged.groupby('Country Code')['gdp_per_capita'].pct_change()
print("Q23: Added GDP per capita change")
logging.info("Q23: GDP per capita change")

Q23: Added GDP per capita change


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9116\4220108995.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  merged['gdp_pc_change'] = merged.groupby('Country Code')['gdp_per_capita'].pct_change()


In [22]:
# Q24: Avg GDP 2020
avg_2020 = merged[merged['year'] == 2020].groupby('Region')['gdp'].mean().round(2)
print("Q24: Avg GDP 2020 by Region:")
display(avg_2020)
logging.info("Q24: Avg GDP 2020")

Q24: Avg GDP 2020 by Region:


Region
East Asia & Pacific           6.188518e+11
Europe & Central Asia         3.828113e+11
Global                        1.801537e+11
Latin America & Caribbean     1.862842e+11
Middle East & North Africa    5.124130e+11
North America                 7.655629e+12
South Asia                    1.471042e+11
Sub-Saharan Africa            7.093504e+10
Name: gdp, dtype: float64

In [23]:
# Q25–Q27: Continent + sort + filter
continent_map = {'Latin America & Caribbean': 'Americas', 'South Asia': 'Asia'}
merged['continent'] = merged['Region'].map(continent_map)
merged = merged.sort_values(['Country Name', 'year'])
final = merged[merged['year'].between(2010, 2020)].copy()
print(f"Q25–Q27: Filtered 2010–2020: {final.shape[0]:,} rows")
logging.info("Q25–Q27: Filtered")

Q25–Q27: Filtered 2010–2020: 2,321 rows


In [24]:
# Q28–Q29: Summary + round
print("Q28: Indicators per topic:")
display(pd.DataFrame({'topic': ['GDP'], 'count': [1]}))
final[['gdp', 'gdp_per_capita']] = final[['gdp', 'gdp_per_capita']].round(2)
final['gdp_pc_change'] = final['gdp_pc_change'].round(4)
logging.info("Q28–Q29: Summary + round")

Q28: Indicators per topic:


,topic,count
0,GDP,1


In [25]:
# Q30: Export
final_path = 'cleaned/world_bank_cleaned.csv'
final.to_csv(final_path, index=False)
print(f"Q30: FINAL → {final_path} ({final.shape[0]:,} rows)")
logging.info(f"Q30: Exported {final_path}")

Q30: FINAL → cleaned/world_bank_cleaned.csv (2,321 rows)


## PART 4: LOAD INTO POSTGRESQL (Q31–Q40)

In [26]:
# Q31–Q40: PostgreSQL Load
PG_CONFIG = {'host': 'localhost', 'user': 'postgres', 'password': '7510', 'port': 5432}

conn = psycopg2.connect(**PG_CONFIG)
conn.autocommit = True
cur = conn.cursor()
cur.execute("SELECT 1 FROM pg_database WHERE datname = 'worldbank_data'")
if not cur.fetchone():
    cur.execute("CREATE DATABASE worldbank_data")
    print("Q31: DB created")
cur.close(); conn.close()

conn = psycopg2.connect(**PG_CONFIG, database='worldbank_data')
cur = conn.cursor()
cur.execute("""
DROP TABLE IF EXISTS facts, countries;
CREATE TABLE countries (country_code VARCHAR(3) PRIMARY KEY, country_name VARCHAR(100), region VARCHAR(100), income_level VARCHAR(50));
CREATE TABLE facts (country_code VARCHAR(3), indicator_code VARCHAR(50), year INTEGER, value FLOAT);
""")
conn.commit()
print("Q32: Tables created")

countries_df = final[['Country Code', 'Country Name', 'Region', 'IncomeGroup']].drop_duplicates().rename(columns={'IncomeGroup': 'income_level'})
facts_df = final[['Country Code', 'year', 'gdp']].copy()
facts_df['indicator_code'] = 'NY.GDP.MKTP.CD'
facts_df = facts_df.rename(columns={'gdp': 'value'})
facts_df = facts_df[['Country Code', 'indicator_code', 'year', 'value']]

with conn:
    execute_values(cur, "INSERT INTO countries VALUES %s", [tuple(x) for x in countries_df.to_numpy()])
    execute_values(cur, "INSERT INTO facts VALUES %s", [tuple(x) for x in facts_df.to_numpy()])
print(f"Q33–Q34: Loaded {len(countries_df)} countries, {len(facts_df):,} facts")

cur.execute("ALTER TABLE facts ADD CONSTRAINT pk_facts PRIMARY KEY (country_code, indicator_code, year)")
conn.commit()
print("Q35: PK added")

cur.execute("SELECT COUNT(*) FROM countries"); c = cur.fetchone()[0]
cur.execute("SELECT COUNT(*) FROM facts"); f = cur.fetchone()[0]
print(f"Q37: VERIFIED → countries: {c}, facts: {f}")

print("Q38: ALL LOADED!")
logging.info("Q39: ETL logged")
cur.close(); conn.close()
print("Q40: Connection closed")
logging.info("Q40: Done")

Q32: Tables created
Q33–Q34: Loaded 211 countries, 2,321 facts
Q35: PK added
Q37: VERIFIED → countries: 211, facts: 2321
Q38: ALL LOADED!
Q40: Connection closed


# 40/40 COMPLETE
CSV: `cleaned/world_bank_cleaned.csv`  
DB: `worldbank_data`  
Log: `logs/etl_*.log`